# KNOWLEDGE

In [ ]:
import os

from google.colab import userdata

os.environ['GEMINI_API_KEY'] =  userdata.get('GOOGLE_API_KEY')


In [ ]:
!pip install -Uq crewai crewai-tools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.0/559.0 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 24.3 MB/s

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from crewai.flow.flow import Flow, start, listen


class MyFlow(Flow):

  @start()
  def function1(self):
    print("step1")

  @listen(function1)
  def function2(self):
    print("step2")

obj = MyFlow()
obj.kickoff()

 
[2025-03-15 16:11:32][🌊 FLOW CREATED: 'MYFLOW']: 2025-03-15 16:11:32.664253
 
[2025-03-15 16:11:32][🤖 FLOW STARTED: 'MYFLOW', 13C32E5A-D244-4F67-B4FA-A91BB3A4906F]: 2025-03-15 16:11:32.666303
 Flow started with ID: 13c32e5a-d244-4f67-b4fa-a91bb3a4906f
 
[2025-03-15 16:11:32][🤖 FLOW METHOD STARTED: 'FUNCTION1']: 2025-03-15 16:11:32.667329
step1
 
[2025-03-15 16:11:32][👍 FLOW METHOD FINISHED: 'FUNCTION1']: 2025-03-15 16:11:32.667448
 
[2025-03-15 16:11:32][🤖 FLOW METHOD STARTED: 'FUNCTION2']: 2025-03-15 16:11:32.667917
step2
 
[2025-03-15 16:11:32][👍 FLOW METHOD FINISHED: 'FUNCTION2']: 2025-03-15 16:11:32.668004
 
[2025-03-15 16:11:32][👍 FLOW FINISHED: 'MYFLOW', 13C32E5A-D244-4F67-B4FA-A91BB3A4906F]: 2025-03-15 16:11:32.668181


In [ ]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
)

llm1.call("What is the capital of France?")

 
[2025-03-15 16:11:32][🤖 LLM CALL STARTED]: 2025-03-15 16:11:32.684202
 
[2025-03-15 16:11:33][✅ LLM CALL COMPLETED]: 2025-03-15 16:11:33.124449


'The capital of France is **Paris**.\n'

In [ ]:
google_embedder = {
    "provider": "google",
    "config": {
         "model": "models/text-embedding-004",
         "api_key": userdata.get('GOOGLE_API_KEY'),
         }
}

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Create a knowledge source
content = "Users name is Muhammad Qasim. He is 30 years old and lives in Karchi, Pakistan. He is working as Chief Data Scientist at CancerClarity LLC"
string_source = StringKnowledgeSource(
    content=content,
)


# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    memory=True,
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source], # Enable knowledge by adding the sources here. You can also add more sources to the sources list.
    embedder=google_embedder

)

result = crew.kickoff(inputs={"question": "What city does Muhammad Qasim live in and how old is he?"})

 
[2025-03-15 16:11:40][🚀 CREW 'CREW' STARTED, 0C8C7F22-D1EF-4ED1-AB84-CD8BC102081C]: 2025-03-15 16:11:40.866837
 
[2025-03-15 16:11:40][📋 TASK STARTED: ANSWER THE FOLLOWING QUESTIONS ABOUT THE USER: WHAT CITY DOES MUHAMMAD QASIM LIVE IN AND HOW OLD IS HE?]: 2025-03-15 16:11:40.882247
 
[2025-03-15 16:11:43][🤖 AGENT 'ABOUT USER' STARTED TASK]: 2025-03-15 16:11:43.103372
# Agent: About User
## Task: Answer the following questions about the user: What city does Muhammad Qasim live in and how old is he?
 
[2025-03-15 16:11:43][🤖 LLM CALL STARTED]: 2025-03-15 16:11:43.103774
 
[2025-03-15 16:11:44][✅ LLM CALL COMPLETED]: 2025-03-15 16:11:44.050550


# Agent: About User
## Final Answer: 
Muhammad Qasim lives in Karachi, Pakistan and he is 30 years old.


 
[2025-03-15 16:12:01][✅ AGENT 'ABOUT USER' COMPLETED TASK]: 2025-03-15 16:12:01.722726
 
[2025-03-15 16:12:01][✅ TASK COMPLETED: ANSWER THE FOLLOWING QUESTIONS ABOUT THE USER: WHAT CITY DOES MUHAMMAD QASIM LIVE IN AND HOW OLD IS HE?]: 202

# MEMORY

In [ ]:
import os
os.environ['CREWAI_STORAGE_DIR']='/my_crew1'

In [ ]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
)

In [ ]:
from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
# from crewai.memory.storage import LTMSQLiteStorage, RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage

from typing import List, Optional

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    memory = True,
    # Long-term memory for persistent storage across sessions
    long_term_memory = LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="./my_crew2/long_term_memory_storage1.db"
        )
    ),
    # Short-term memory for current context using RAG
    short_term_memory = ShortTermMemory(
        storage = RAGStorage(
                embedder_config=google_embedder,
                type="short_term",
                path="./my_crew2/short_term1/"
            )
        ),

    # Entity memory for tracking key information about entities
    entity_memory = EntityMemory(
        storage=RAGStorage(
            embedder_config=google_embedder,
            type="short_term",
            path="./my_crew2/entity1/"
        )
    ),
    verbose=True,
)

# Integrating Mem0 for Enhanced User Memory


In [ ]:
import os
from crewai import Crew, Process
from mem0 import MemoryClient

# Set environment variables for Mem0
os.environ["MEM0_API_KEY"] = userdata.get('MEM0_API_KEY')

# Step 1: Record preferences based on past conversation or user input
client = MemoryClient()
# messages = [
#     {"role": "user", "content": "Hi there! I'm planning a vacation and could use some advice."},
#     {"role": "assistant", "content": "Hello! I'd be happy to help with your vacation planning. What kind of destination do you prefer?"},
#     {"role": "user", "content": "I am more of a beach person than a mountain person."},
#     {"role": "assistant", "content": "That's interesting. Do you like hotels or Airbnb?"},
#     {"role": "user", "content": "I like Airbnb more."},
# ]

# client.add(messages, user_id="john", output_format="v1.1")

# Step 2: Create a Crew with User Memory

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
    memory=True
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew1 = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    verbose=True,
    memory=True,
    memory_config={
        "provider": "mem0",
        "config": {"user_id": "john"},
    },
)

crew1.kickoff(inputs={"question": "What is john's favorite vacation destination?"})

 
[2025-03-15 16:12:06][🚀 CREW 'CREW' STARTED, 1822EBF1-C08A-4692-81FD-3B9DBA2593B9]: 2025-03-15 16:12:06.717427
 
[2025-03-15 16:12:06][📋 TASK STARTED: ANSWER THE FOLLOWING QUESTIONS ABOUT THE USER: WHAT IS JOHN'S FAVORITE VACATION DESTINATION?]: 2025-03-15 16:12:06.731475
 
[2025-03-15 16:12:06][❌ TASK FAILED: ANSWER THE FOLLOWING QUESTIONS ABOUT THE USER: WHAT IS JOHN'S FAVORITE VACATION DESTINATION?]: 2025-03-15 16:12:06.892748
 
[2025-03-15 16:12:06][❌ CREW 'CREW' FAILED, 1822EBF1-C08A-4692-81FD-3B9DBA2593B9]: 2025-03-15 16:12:06.892881


AttributeError: 'NoneType' object has no attribute 'search'

# Tools

In [ ]:
!pip install -Uq 'crewai[tools]'

In [ ]:
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [ ]:
# !pip install -Uq langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI

llm1=LLM(model="gemini/gemini-1.5-flash")

In [ ]:
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY') # serper.dev API key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()

web_rag_tool = WebsiteSearchTool(
    config=dict(
        llm=dict(
            provider="google", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gemini/gemini-2.0-flash",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)


# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True,
    llm=llm1
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True,
    llm=llm1
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst’s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,  # Enable planning feature
    planning_llm=llm1
)

# Execute tasks
crew.kickoff()

 
[2025-03-15 17:09:27][🚀 CREW 'CREW' STARTED, B5B32FE2-D246-4820-A03B-90E3B1C0CA34]: 2025-03-15 17:09:27.318212
 
[2025-03-15 17:09:27][INFO]: Planning the crew execution
 
[2025-03-15 17:09:27][📋 TASK STARTED: BASED ON THESE TASKS SUMMARY: 
                TASK NUMBER 1 - RESEARCH THE LATEST TRENDS IN THE AI INDUSTRY AND PROVIDE A SUMMARY.
                "TASK_DESCRIPTION": RESEARCH THE LATEST TRENDS IN THE AI INDUSTRY AND PROVIDE A SUMMARY.
                "TASK_EXPECTED_OUTPUT": A SUMMARY OF THE TOP 3 TRENDING DEVELOPMENTS IN THE AI INDUSTRY WITH A UNIQUE PERSPECTIVE ON THEIR SIGNIFICANCE.
                "AGENT": MARKET RESEARCH ANALYST
                "AGENT_GOAL": PROVIDE UP-TO-DATE MARKET ANALYSIS OF THE AI INDUSTRY
                "TASK_TOOLS": [SERPERDEVTOOL(NAME='SEARCH THE INTERNET WITH SERPER', DESCRIPTION="TOOL NAME: SEARCH THE INTERNET WITH SERPER\NTOOL ARGUMENTS: {'SEARCH_QUERY': {'DESCRIPTION': 'MANDATORY SEARCH QUERY YOU WANT TO USE TO SEARCH THE INTERNET', 'TYPE': '

CrewOutput(raw="## AI: A World Transformed\n\nThe AI revolution is unfolding at breakneck speed, impacting nearly every facet of our lives.  From self-driving cars to medical diagnoses, AI's influence is undeniable.  But this rapid advancement isn't just about technological progress; it's about navigating ethical complexities and shaping a future where AI serves humanity's best interests.\n\nGenerative AI, with its ability to create realistic text, images, and code, has exploded onto the scene. Businesses are rapidly adopting these tools to boost productivity and unlock new creative possibilities.  However,  this rapid growth necessitates a careful consideration of copyright issues, ethical implications, and the potential for misuse in spreading misinformation.  We must prioritize responsible development and deployment, balancing innovation with robust regulatory frameworks.\n\nThe ethical considerations surrounding AI are no longer just theoretical discussions; they're becoming the fo

In [ ]:
# prompt: import markdown for jupyter display

from IPython.display import Markdown

def display_markdown_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    display(Markdown(content))

# Example usage: Assuming 'blog-posts/new_post.md' is the file generated by your code
display_markdown_file('blog-posts/new_post.md')


## AI: A World Transformed

The AI revolution is unfolding at breakneck speed, impacting nearly every facet of our lives.  From self-driving cars to medical diagnoses, AI's influence is undeniable.  But this rapid advancement isn't just about technological progress; it's about navigating ethical complexities and shaping a future where AI serves humanity's best interests.

Generative AI, with its ability to create realistic text, images, and code, has exploded onto the scene. Businesses are rapidly adopting these tools to boost productivity and unlock new creative possibilities.  However,  this rapid growth necessitates a careful consideration of copyright issues, ethical implications, and the potential for misuse in spreading misinformation.  We must prioritize responsible development and deployment, balancing innovation with robust regulatory frameworks.

The ethical considerations surrounding AI are no longer just theoretical discussions; they're becoming the foundation of tangible regulations.  Governments worldwide are grappling with how to balance fostering AI innovation with mitigating its potential risks. This nascent regulatory landscape will play a pivotal role in shaping the future of AI, impacting investment, adoption rates, and the overall development of the technology.  Striking a balance between fostering innovation and responsible AI use will be a critical challenge moving forward.

AI's transformative power extends far beyond tech giants.  It's becoming increasingly accessible to smaller businesses and individuals, leading to innovative applications across all sectors.  This democratization of AI empowers individuals and communities to leverage its potential, driving efficiency and creating new opportunities.  However, we must strive for equitable access to these technologies to prevent the exacerbation of existing inequalities, ensuring that AI's benefits are shared broadly.  The future will require substantial investment in education and workforce development to maximize AI's positive societal impact.

## Creating your own Tools

In [ ]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

class MyToolInput(BaseModel):
    """Input schema for MyCustomTool."""
    student_name: str = Field(..., description="Student name")
    student_roll_no : int = Field(..., description="student id")

class PiaicStudentCard(BaseTool):
    name: str = "Piaic student card generator"
    description: str = "this function will create Piaic student card"
    args_schema: Type[BaseModel] = MyToolInput

    def _run(self, student_name: str, student_roll_no: int ) -> str:
        # Your tool's logic here
        return f"""PIAIC student card
student name: {student_name}
student roll no: {student_roll_no}
Pakistan zindabd!
        """

In [ ]:
from crewai.tools import tool

@tool("PIAIC fee update")
def my_tool(roll_no: int) -> dict | str:
    """this function search piaic student fee updates, it will required roll no of PIAIC student"""
    #database

    data = {100:'paid',
         200:'unpaid'}


    status = data.get(roll_no)

    if status:
      return {"status": status}
    else:
      return "student not found"





In [ ]:
from crewai import Agent, Task, Crew

card = PiaicStudentCard()


piaic_manager = Agent(
    role="PIAIC manager",
    goal = "Manage all quries regarding PIAIC and you will use only relevant tools for student query",
    backstory="""You are a master at understanding people and their preferences.""",
    tools=[card, my_tool],
    verbose=True,
    llm=llm1
)

piaic_card_creator = Task(
    description="you will be responsible for all PIAIC relevant operations, student query '{query}' you must be know how to answer his question based on final context",
    expected_output="final query answer only",
    agent=piaic_manager
)

crew = Crew(
    agents=[piaic_manager],
    tasks=[piaic_card_creator],
    verbose=True
)

result = crew.kickoff(inputs={
    "query":"I'm PIAIC student my name is Muhammad Qasim and my roll number is 100, can you create my student card."
})

print(result)

 
[2025-03-15 17:50:38][🚀 CREW 'CREW' STARTED, 9C502162-E51C-4846-9AA6-3DC862E2A5E8]: 2025-03-15 17:50:38.277541
 
[2025-03-15 17:50:38][📋 TASK STARTED: YOU WILL BE RESPONSIBLE FOR ALL PIAIC RELEVANT OPERATIONS, STUDENT QUERY 'I'M PIAIC STUDENT MY NAME IS MUHAMMAD QASIM AND MY ROLL NUMBER IS 100, CAN YOU CREATE MY STUDENT CARD.' YOU MUST BE KNOW HOW TO ANSWER HIS QUESTION BASED ON FINAL CONTEXT]: 2025-03-15 17:50:38.300519
 
[2025-03-15 17:50:38][🤖 AGENT 'PIAIC MANAGER' STARTED TASK]: 2025-03-15 17:50:38.302330
# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student my name is Muhammad Qasim and my roll number is 100, can you create my student card.' you must be know how to answer his question based on final context
 
[2025-03-15 17:50:38][🤖 LLM CALL STARTED]: 2025-03-15 17:50:38.302701
 
[2025-03-15 17:50:40][✅ LLM CALL COMPLETED]: 2025-03-15 17:50:40.065638
 
[2025-03-15 17:50:40][🤖 TOOL USAGE STARTED: 'PIAIC STUDENT

In [ ]:
result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my roll number is 100, can you share my fee updates?."
})

print(result)

 
[2025-03-15 17:50:50][🚀 CREW 'CREW' STARTED, 9C502162-E51C-4846-9AA6-3DC862E2A5E8]: 2025-03-15 17:50:50.195857
 
[2025-03-15 17:50:50][📋 TASK STARTED: YOU WILL BE RESPONSIBLE FOR ALL PIAIC RELEVANT OPERATIONS, STUDENT QUERY 'I'M PIAIC STUDENT  MY ROLL NUMBER IS 100, CAN YOU SHARE MY FEE UPDATES?.' YOU MUST BE KNOW HOW TO ANSWER HIS QUESTION BASED ON FINAL CONTEXT]: 2025-03-15 17:50:50.214411
 
[2025-03-15 17:50:50][🤖 AGENT 'PIAIC MANAGER' STARTED TASK]: 2025-03-15 17:50:50.216203
# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my roll number is 100, can you share my fee updates?.' you must be know how to answer his question based on final context
 
[2025-03-15 17:50:50][🤖 LLM CALL STARTED]: 2025-03-15 17:50:50.216454
 
[2025-03-15 17:50:51][✅ LLM CALL COMPLETED]: 2025-03-15 17:50:51.868608
 
[2025-03-15 17:50:51][🤖 TOOL USAGE STARTED: 'PIAIC FEE UPDATE']: 2025-03-15 17:50:51.869686
 
[2025-03-15 17:50:51][✅ 

In [ ]:
result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my roll number is 200, can you share my fee updates?."
})

print(result)

 
[2025-03-15 17:50:58][🚀 CREW 'CREW' STARTED, 9C502162-E51C-4846-9AA6-3DC862E2A5E8]: 2025-03-15 17:50:58.595060
 
[2025-03-15 17:50:58][📋 TASK STARTED: YOU WILL BE RESPONSIBLE FOR ALL PIAIC RELEVANT OPERATIONS, STUDENT QUERY 'I'M PIAIC STUDENT  MY ROLL NUMBER IS 200, CAN YOU SHARE MY FEE UPDATES?.' YOU MUST BE KNOW HOW TO ANSWER HIS QUESTION BASED ON FINAL CONTEXT]: 2025-03-15 17:50:58.614724
 
[2025-03-15 17:50:58][🤖 AGENT 'PIAIC MANAGER' STARTED TASK]: 2025-03-15 17:50:58.616506
# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my roll number is 200, can you share my fee updates?.' you must be know how to answer his question based on final context
 
[2025-03-15 17:50:58][🤖 LLM CALL STARTED]: 2025-03-15 17:50:58.616798
 
[2025-03-15 17:50:59][✅ LLM CALL COMPLETED]: 2025-03-15 17:50:59.809189
 
[2025-03-15 17:50:59][🤖 TOOL USAGE STARTED: 'PIAIC FEE UPDATE']: 2025-03-15 17:50:59.809512
 
[2025-03-15 17:50:59][✅ 

# Process

In [ ]:
import os
from crewai import Agent, Task, Crew, Process

# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)



# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts2')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool(
    config=dict(
        llm=dict(
            provider="google", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gemini/gemini-2.0-flash",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)


# Define your agents
researcher = Agent(
    role="Researcher",
    goal="Conduct thorough research and analysis on AI and AI agents",
    backstory="You're an expert researcher, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently researching for a new client.",
    allow_delegation=False,
    tools=[search_tool,web_rag_tool],
    llm=llm1
)

writer = Agent(
    role="Senior Writer",
    goal="Create compelling content about AI and AI agents",
    backstory="You're a senior writer, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently writing content for a new client.",
    allow_delegation=False,
    tools=[docs_tool,file_tool],
    llm=llm1
)

# Define your task
task = Task(
    description="Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.",
    expected_output="5 bullet points, each with a paragraph and accompanying notes.",
)

# Define the manager agent
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
    llm=llm1
)

# Instantiate your crew with a custom manager
crew = Crew(
    agents=[researcher, writer],
    tasks=[task],
    manager_agent=manager,
    function_calling_llm=llm1,
    process=Process.hierarchical,
    verbose=True

)

# Start the crew's work
result = crew.kickoff()
from IPython.display import Markdown

Markdown(result.raw)

 
[2025-03-15 18:01:13][🚀 CREW 'CREW' STARTED, 51C9C8D6-B926-4291-B8E6-8043EDE664BF]: 2025-03-15 18:01:13.683813
 
[2025-03-15 18:01:13][📋 TASK STARTED: GENERATE A LIST OF 5 INTERESTING IDEAS FOR AN ARTICLE, THEN WRITE ONE CAPTIVATING PARAGRAPH FOR EACH IDEA THAT SHOWCASES THE POTENTIAL OF A FULL ARTICLE ON THIS TOPIC. RETURN THE LIST OF IDEAS WITH THEIR PARAGRAPHS AND YOUR NOTES.]: 2025-03-15 18:01:13.701428
 
[2025-03-15 18:01:13][🤖 AGENT 'PROJECT MANAGER' STARTED TASK]: 2025-03-15 18:01:13.702722
# Agent: Project Manager
## Task: Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.
 
[2025-03-15 18:01:13][🤖 LLM CALL STARTED]: 2025-03-15 18:01:13.702920
 
[2025-03-15 18:01:26][✅ LLM CALL COMPLETED]: 2025-03-15 18:01:26.148372
 
[2025-03-15 18:01:26][🤖 TOOL USAGE STARTED: 'DELEGATE WORK TO COWORKER']: 2025-03-

* **Idea 1: The Unexpected Benefits of Learning a New Language in Adulthood**
   *Paragraph:*  While often associated with childhood, learning a new language as an adult offers a surprising array of cognitive benefits, extending beyond simple communication.  From boosting memory and improving multitasking abilities to delaying the onset of age-related cognitive decline, the impact on the brain is remarkable. A full article would explore the scientific evidence behind these benefits, examine the most effective learning methods for adults, and address the common challenges and rewards of embracing linguistic diversity later in life.
   *Notes:*  Focus on scientific studies and practical tips for adult learners. Include diverse perspectives and success stories.

* **Idea 2: The Ethics of Artificial Intelligence in Everyday Life**
   *Paragraph:* The rapid advancement of artificial intelligence is transforming our daily lives, from the algorithms that curate our social media feeds to the self-driving cars on our roads. But these technological marvels raise complex ethical questions:  How do we ensure fairness and prevent bias in AI systems?  What are the implications for privacy and employment?  Exploring these dilemmas is crucial as we navigate a future increasingly shaped by AI. A full article would delve into these ethical challenges, examining real-world case studies and proposing potential solutions.
   *Notes:*  Focus on concrete examples and real-world scenarios. Include expert opinions from ethicists and technologists.

* **Idea 3: The Power of Micro-Habits: Small Changes, Big Results**
   *Paragraph:*  We often set ambitious goals, only to find ourselves overwhelmed and unable to sustain the effort. The secret to lasting change lies not in grand gestures, but in the accumulation of tiny, consistent actions. Micro-habits, small changes implemented daily, can create a ripple effect, leading to significant improvements in productivity, well-being, and overall life satisfaction.  A full article would explore the science of habit formation, offer practical examples of effective micro-habits, and provide strategies for building lasting positive routines.
   *Notes:* Focus on actionable strategies and practical tips. Include case studies and success stories.

* **Idea 4: The Future of Work: Adapting to a Changing Landscape**
   *Paragraph:* The nature of work is undergoing a radical transformation, driven by automation, globalization, and the rise of the gig economy.  To thrive in this evolving landscape, individuals and businesses need to adapt and acquire new skills.  A full article would explore the key trends shaping the future of work, examine the implications for various professions, and provide insights on how to develop the skills and adaptability needed to succeed in the modern workplace.
   *Notes:*  Focus on future trends and their impact on various job sectors.  Include insights from futurists and industry experts.

* **Idea 5: The Art of Minimalism: Decluttering Your Life for Greater Well-being**
   *Paragraph:* In a culture of excess, minimalism offers a powerful antidote to the stress and overwhelm of modern life.  By simplifying our possessions and our lives, we can cultivate greater clarity, focus, and a deeper sense of purpose.  A full article would explore the philosophy of minimalism, offer practical advice on decluttering and simplifying various aspects of life, and examine the positive impact it can have on mental and emotional well-being.
   *Notes:* Focus on actionable steps and practical tips.  Include personal stories and success stories of those who have embraced minimalism.

# Advance Tools

In [ ]:
!pip install databricks-sdk

In [ ]:


from pydantic import BaseModel, Field

class TeacherAssignmentOutPut(BaseModel):
  score :int  = Field(..., description="student assignment score")
  feedback : str = Field(..., description="teacher feedback")
  question : str = Field(..., description="question")
  answer : str = Field(..., description="python code for markdown format")

In [ ]:

from crewai import Agent
from crewai_tools import CodeInterpreterTool
from crewai import Task, Crew, Process

teacher = Agent(
    role="AI teacher",
    goal="you are Agentic AI teacher you have to check python assignment code submission.",
    backstory="you have to check student python code submission.",
    tools=[CodeInterpreterTool()],
    llm=llm1
)

assignment_check = Task(
    description="you are Agentic AI teacher you have to check python assignment code submission. question: '''{qestion}''' student solution : '''{solution}''' ",
    expected_output="final code is runing, assign number between 1-10",
    output_pydantic=TeacherAssignmentOutPut,
    agent=teacher # Added the missing agent here
)


crew = Crew(
    agents=[teacher],
    tasks=[assignment_check],
    process=Process.sequential,
    verbose=True,
)

question = "create function add two numbers"

solution = """
def add_two_number(num1 : int , num2 : int):
  return

"""

result = crew.kickoff(inputs={
    "qestion": question,
    "solution" : solution
})

print(result)

 
[2025-03-15 16:27:22][🚀 CREW 'CREW' STARTED, 20857FC6-1EE6-4FDE-9ED7-B41AA730110E]: 2025-03-15 16:27:22.516848
 
[2025-03-15 16:27:22][📋 TASK STARTED: YOU ARE AGENTIC AI TEACHER YOU HAVE TO CHECK PYTHON ASSIGNMENT CODE SUBMISSION. QUESTION: '''CREATE FUNCTION ADD TWO NUMBERS''' STUDENT SOLUTION : '''
DEF ADD_TWO_NUMBER(NUM1 : INT , NUM2 : INT):
  RETURN

''' ]: 2025-03-15 16:27:22.532908
 
[2025-03-15 16:27:22][🤖 AGENT 'AI TEACHER' STARTED TASK]: 2025-03-15 16:27:22.534111
# Agent: AI teacher
## Task: you are Agentic AI teacher you have to check python assignment code submission. question: '''create function add two numbers''' student solution : '''
def add_two_number(num1 : int , num2 : int):
  return

''' 
 
[2025-03-15 16:27:22][🤖 LLM CALL STARTED]: 2025-03-15 16:27:22.534303
 
[2025-03-15 16:27:24][✅ LLM CALL COMPLETED]: 2025-03-15 16:27:24.778915
 
[2025-03-15 16:27:24][🤖 TOOL USAGE STARTED: 'CODE INTERPRETER']: 2025-03-15 16:27:24.779362
 
[2025-03-15 16:27:24][❌ TOOL USAGE ERR

In [ ]:
result.to_dict()

{'score': 6,
 'feedback': 'The function definition is good, including type hinting. However, the function body is incomplete. It should return the sum of the two input numbers. Also consider adding some test cases to ensure your function is working correctly.',
 'question': 'create function add two numbers',
 'answer': 'def add_two_number(num1 : int , num2 : int):\n  return num1 + num2'}

In [ ]:
for k,v in result.to_dict().items():
  print(str("*"*10) + str(k) + str("*"*10))
  display(v)

**********score**********


9

**********feedback**********


"The function correctly adds two numbers.  Consider adding docstrings for better readability and to explain the function's purpose."

**********question**********


'create function add two numbers'

**********answer**********


'def add_two_number(num1, num2):\n  return num1 + num2\n'